In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings, sys

pd.set_option('display.max_columns', 100)
warnings.filterwarnings('ignore')
%matplotlib inline

In [7]:
weather = pd.read_csv('../data/vzw.csv')
env = pd.read_csv('../data/vz311/vz311intersectionsLighter.txt', delimiter='\t', encoding='utf8', engine='python')
env.columns = [i.replace(' ', '_') for i in env.columns]
weather['UNIQUE KEY'] = weather['UNIQUE KEY'].fillna(0)
weather['UNIQUE KEY'] = weather['UNIQUE KEY'].astype(str)
env['UNIQUE_KEY'] = env.UNIQUE_KEY.fillna(1)
env['UNIQUE_KEY'] = env.UNIQUE_KEY.astype(str)
# minute index
# env['time_stamp'] = pd.to_datetime(env.TIME, format='%H:%m:%s')
# env['time_idx'] = env.TIME.apply(lambda x: str(x).split(':')[1])
# weather['time_idx'] = weather.TIME.apply(lambda x: str(x).split(':')[-1])

In [38]:
# weather
# weather.isnull().sum()
# env.shape

(277255, 35)

In [3]:
# env.head()
# weather=None
# env=None

In [8]:
weather['UNIQUE KEY'].value_counts().reset_index().sort('index').head()

,index,UNIQUE KEY
0,0.0,15966
97583,3145850.0,1
176544,3145851.0,1
48757,3145852.0,1
118244,3145853.0,1


In [9]:
env.UNIQUE_KEY.value_counts().reset_index().sort('index').head()

,index,UNIQUE_KEY
0,1.0,94458
4262,3145850.0,1
69205,3145851.0,1
101330,3145852.0,1
166025,3145853.0,1


In [10]:
# assign labels
env['label'] = np.where(env.DATE.isnull(), 0, 1)

# change the nans to 0s
env['NUMBER_OF_CYCLIST_INJURED'] = np.where(env.NUMBER_OF_CYCLIST_INJURED.isnull(), 0, env.NUMBER_OF_CYCLIST_INJURED)
env['NUMBER_OF_CYCLIST_KILLED'] = np.where(env.NUMBER_OF_CYCLIST_KILLED.isnull(), 0, env.NUMBER_OF_CYCLIST_KILLED)
env['NUMBER_OF_MOTORIST_INJURED'] = np.where(env.NUMBER_OF_MOTORIST_INJURED.isnull(), 0, env.NUMBER_OF_MOTORIST_INJURED)
env['NUMBER_OF_MOTORIST_KILLED'] = np.where(env.NUMBER_OF_MOTORIST_KILLED.isnull(), 0, env.NUMBER_OF_MOTORIST_KILLED)
env['NUMBER_OF_PEDESTRIANS_INJURED'] = np.where(env.NUMBER_OF_PEDESTRIANS_INJURED.isnull(), 0, env.NUMBER_OF_PEDESTRIANS_INJURED)
env['NUMBER_OF_PEDESTRIANS_KILLED'] = np.where(env.NUMBER_OF_PEDESTRIANS_KILLED.isnull(), 0, env.NUMBER_OF_PEDESTRIANS_KILLED)
env['NUMBER_OF_PERSONS_INJURED'] = np.where(env.NUMBER_OF_PERSONS_INJURED.isnull(), 0, env.NUMBER_OF_PERSONS_INJURED)
env['NUMBER_OF_PERSONS_KILLED'] = np.where(env.NUMBER_OF_PERSONS_KILLED.isnull(), 0, env.NUMBER_OF_PERSONS_KILLED)

# assign speical vehicle code type for nans
codes = np.arange(1,6, dtype=np.int64)
for i in codes:
    car_type = 'VEHICLE_TYPE_CODE_%d' % i
    env[car_type] = np.where(env[car_type].isnull(), 'CONTROL', env[car_type])
    
# assign special contributing factor for nans
for j in codes:
    fac_type = 'CONTRIBUTING_FACTOR_VEHICLE_%d' % j
    env[fac_type] = np.where(env[fac_type].isnull(), 'CONTROL', env[fac_type])

In [11]:
df = env.merge(weather, how='left', left_on=['UNIQUE_KEY'], right_on=['UNIQUE KEY'])

In [43]:
# weather2.head()
# env.head()
# df.isnull().sum()

In [44]:
df.shape

(277255, 71)

### call the full weather dataset and randomly fill the NAN in the merged df

In [45]:
fw = pd.read_csv('../data/weather_raw_NYC/full_weather.csv') 

In [46]:
np.random.seed(83)
scores = np.random.random_integers(0, fw.shape[0]-1, size=df.DATE_x.isnull().sum())

In [47]:
eidx = df[df.DATE_x.isnull()].index.values

In [48]:
gg = fw.iloc[scores]

In [49]:
gg['new_index'] = eidx
gg.columns = ['new_%s' % j for j in gg.columns]

In [50]:
df2 = df.merge(gg, how='left', left_index=True, right_on='new_new_index')

In [51]:
df2.shape

(277255, 79)

In [52]:
# replace the nan values for null dates in full file
df2['Visibility'] = np.where(df2.DATE_x.isnull(), df2.new_Visibility, df2.Visibility)
df2['WetBulbFarenheit'] = np.where(df2.DATE_x.isnull(), df2.new_WetBulbFarenheit, df2.WetBulbFarenheit)
df2['WindSpeed'] = np.where(df2.DATE_x.isnull(), df2.new_WindSpeed, df2.WindSpeed)
df2['Precip'] = np.where(df2.DATE_x.isnull(), df2.new_Precip, df2.Precip)
df2['PrecipSum'] = np.where(df2.DATE_x.isnull(), df2.new_PrecipSum, df2.PrecipSum)

In [53]:
df2.shape, df.shape

((277255, 79), (277255, 71))

In [20]:
# for i, v in enumerate(eidx):
#     sys.stdout.write('\rline %d' % i)
#     sys.stdout.flush()
#     idx = scores[i]
#     ww = fw.iloc[idx]
#     df.iloc[v]['DATE'] = ww.Date
#     df.iloc[v]['Visibility'] = ww.Visibility
#     df.iloc[v]['WetBulbFarenheit'] = ww.WetBulbFarenheit
#     df.iloc[v]['WindSpeed'] = ww.WindSpeed
#     df.iloc[v]['Precip'] = ww.Precip
#     df.iloc[v]['PrecipSum'] = ww.PrecipSum

In [54]:
df.shape[0] == env.shape[0], env.shape

(True, (277255, 36))

In [55]:
df.isnull().sum()

index                                 0
VIntersect                       272155
BOROUGH_x                        113934
CONTRIBUTING_FACTOR_VEHICLE_1         0
CONTRIBUTING_FACTOR_VEHICLE_2         0
CONTRIBUTING_FACTOR_VEHICLE_3         0
CONTRIBUTING_FACTOR_VEHICLE_4         0
CONTRIBUTING_FACTOR_VEHICLE_5         0
CROSS_STREET_NAME                116304
DATE_x                            94457
LATITUDE_x                        94456
LOCATION_x                        94456
LONGITUDE_x                       94456
NUMBER_OF_CYCLIST_INJURED             0
NUMBER_OF_CYCLIST_KILLED              0
NUMBER_OF_MOTORIST_INJURED            0
NUMBER_OF_MOTORIST_KILLED             0
NUMBER_OF_PEDESTRIANS_INJURED         0
NUMBER_OF_PEDESTRIANS_KILLED          0
NUMBER_OF_PERSONS_INJURED             0
NUMBER_OF_PERSONS_KILLED              0
OFF_STREET_NAME                  273324
ON_STREET_NAME                   116304
TIME_x                            94458
UNIQUE_KEY                            0


### Read in all the data and give it an index

In [56]:
# df = pd.read_csv('../data/vz_full.csv')
df2['identity'] = ['id-%d' % d for d in np.arange(0, df2.shape[0], dtype=np.int64)]

In [57]:
# subset the columns
cc = ['identity', 'label', 'NUMBER_OF_PERSONS_INJURED', 'NUMBER_OF_PERSONS_KILLED',
      'NUMBER_OF_PEDESTRIANS_INJURED', 'NUMBER_OF_PEDESTRIANS_KILLED','NUMBER_OF_CYCLIST_INJURED', 
      'NUMBER_OF_CYCLIST_KILLED','NUMBER_OF_MOTORIST_INJURED', 'NUMBER_OF_MOTORIST_KILLED',
      'CONTRIBUTING_FACTOR_VEHICLE_1', 'CONTRIBUTING_FACTOR_VEHICLE_2', 'CONTRIBUTING_FACTOR_VEHICLE_3',
      'CONTRIBUTING_FACTOR_VEHICLE_4', 'CONTRIBUTING_FACTOR_VEHICLE_5', 'VEHICLE_TYPE_CODE_1',
      'VEHICLE_TYPE_CODE_2', 'VEHICLE_TYPE_CODE_3', 'VEHICLE_TYPE_CODE_4', 'VEHICLE_TYPE_CODE_5',
      'Street_Condition', 'Traffic_Signal_Condition', 'Visibility', 'WetBulbFarenheit',
      'WindSpeed', 'Precip', 'PrecipSum','intersectionID']

In [68]:
# subset
d1 = df2[cc]

# remove the column spaces with _
d1.columns = [x.replace(' ', '_') for x in d1.columns]

In [69]:
d1.shape

(277255, 28)

In [ ]:
# # take the string out of cyclist value
# d1['NUMBER_OF_CYCLIST_INJURED'] = np.where(d1.NUMBER_OF_CYCLIST_INJURED=='PASSENGER VEHICLE', 
#                                            np.nan, d1.NUMBER_OF_CYCLIST_INJURED).astype(np.float64)
# d1['NUMBER_OF_CYCLIST_KILLED'] = np.where(d1.NUMBER_OF_CYCLIST_KILLED=='PASSENGER VEHICLE', 
#                                           np.nan, d1.NUMBER_OF_CYCLIST_KILLED).astype(np.float64)

In [70]:
# total persons involved in accident [injured + killed]
d1['total_involved'] = np.sum(d1.values[:,1:9].astype(np.float64), axis=1).ravel()

In [3]:
# d1.to_csv('temp_data.csv')
d1 = pd.read_csv('temp_data.csv').drop(['Unnamed: 0'],1)

In [4]:
d1.shape

(277255, 29)

In [5]:
# dummy out the strings
d2 = pd.DataFrame(d1.identity)
for head in d1.columns:
    if head != 'identity':
        if d1[head].dtype == np.float64:
            d2 = d2.merge(pd.DataFrame(d1[head]), left_index=True, right_index=True)
        elif d1[head].dtype == object:
            temp = pd.get_dummies(d1[head], prefix=head, prefix_sep='-', dummy_na=True)
            d2 = d2.merge(temp, right_index=True, left_index=True)
        elif d1[head].dtype == np.int64:
            d2[head] =d1[head]
        else:
            print('Skipped a Column', head)

In [7]:
d2.shape

(277255, 281)

In [10]:
d2.head()

,identity,label,NUMBER_OF_PERSONS_INJURED,NUMBER_OF_PERSONS_KILLED,NUMBER_OF_PEDESTRIANS_INJURED,NUMBER_OF_PEDESTRIANS_KILLED,NUMBER_OF_CYCLIST_INJURED,NUMBER_OF_CYCLIST_KILLED,NUMBER_OF_MOTORIST_INJURED,NUMBER_OF_MOTORIST_KILLED,CONTRIBUTING_FACTOR_VEHICLE_1-3291249.0,CONTRIBUTING_FACTOR_VEHICLE_1-Accelerator Defective,CONTRIBUTING_FACTOR_VEHICLE_1-Aggressive Driving/Road Rage,CONTRIBUTING_FACTOR_VEHICLE_1-Alcohol Involvement,CONTRIBUTING_FACTOR_VEHICLE_1-Animals Action,CONTRIBUTING_FACTOR_VEHICLE_1-Backing Unsafely,CONTRIBUTING_FACTOR_VEHICLE_1-Brakes Defective,CONTRIBUTING_FACTOR_VEHICLE_1-CONTROL,CONTRIBUTING_FACTOR_VEHICLE_1-Cell Phone (hand-held),CONTRIBUTING_FACTOR_VEHICLE_1-Cell Phone (hands-free),CONTRIBUTING_FACTOR_VEHICLE_1-Driver Inattention/Distraction,CONTRIBUTING_FACTOR_VEHICLE_1-Driver Inexperience,CONTRIBUTING_FACTOR_VEHICLE_1-Drugs (Illegal),CONTRIBUTING_FACTOR_VEHICLE_1-Failure to Keep Right,CONTRIBUTING_FACTOR_VEHICLE_1-Failure to Yield Right-of-Way,CONTRIBUTING_FACTOR_VEHICLE_1-Fatigued/Drowsy,CONTRIBUTING_FACTOR_VEHICLE_1-Fell Asleep,CONTRIBUTING_FACTOR_VEHICLE_1-Following Too Closely,CONTRIBUTING_FACTOR_VEHICLE_1-Glare,CONTRIBUTING_FACTOR_VEHICLE_1-Headlights Defective,CONTRIBUTING_FACTOR_VEHICLE_1-Illness,CONTRIBUTING_FACTOR_VEHICLE_1-Lane Marking Improper/Inadequate,CONTRIBUTING_FACTOR_VEHICLE_1-Lost Consciousness,CONTRIBUTING_FACTOR_VEHICLE_1-Obstruction/Debris,CONTRIBUTING_FACTOR_VEHICLE_1-Other Electronic Device,CONTRIBUTING_FACTOR_VEHICLE_1-Other Lighting Defects,CONTRIBUTING_FACTOR_VEHICLE_1-Other Vehicular,CONTRIBUTING_FACTOR_VEHICLE_1-Outside Car Distraction,CONTRIBUTING_FACTOR_VEHICLE_1-Oversized Vehicle,CONTRIBUTING_FACTOR_VEHICLE_1-Passenger Distraction,CONTRIBUTING_FACTOR_VEHICLE_1-Passing or Lane Usage Improper,CONTRIBUTING_FACTOR_VEHICLE_1-Pavement Defective,CONTRIBUTING_FACTOR_VEHICLE_1-Pavement Slippery,CONTRIBUTING_FACTOR_VEHICLE_1-Pedestrian/Bicyclist/Other Pedestrian Error/Confusion,CONTRIBUTING_FACTOR_VEHICLE_1-Physical Disability,CONTRIBUTING_FACTOR_VEHICLE_1-Prescription Medication,CONTRIBUTING_FACTOR_VEHICLE_1-Reaction to Other Uninvolved Vehicle,CONTRIBUTING_FACTOR_VEHICLE_1-Shoulders Defective/Improper,CONTRIBUTING_FACTOR_VEHICLE_1-Steering Failure,CONTRIBUTING_FACTOR_VEHICLE_1-Tire Failure/Inadequate,...,VEHICLE_TYPE_CODE_3-MOTORCYCLE,VEHICLE_TYPE_CODE_3-OTHER,VEHICLE_TYPE_CODE_3-PASSENGER VEHICLE,VEHICLE_TYPE_CODE_3-PEDICAB,VEHICLE_TYPE_CODE_3-PICK-UP TRUCK,VEHICLE_TYPE_CODE_3-SCOOTER,VEHICLE_TYPE_CODE_3-SMALL COM VEH(4 TIRES),VEHICLE_TYPE_CODE_3-SPORT UTILITY / STATION WAGON,VEHICLE_TYPE_CODE_3-TAXI,VEHICLE_TYPE_CODE_3-UNKNOWN,VEHICLE_TYPE_CODE_3-VAN,VEHICLE_TYPE_CODE_3-nan,VEHICLE_TYPE_CODE_4-BICYCLE,VEHICLE_TYPE_CODE_4-BUS,VEHICLE_TYPE_CODE_4-CONTROL,VEHICLE_TYPE_CODE_4-FIRE TRUCK,VEHICLE_TYPE_CODE_4-LARGE COM VEH(6 OR MORE TIRES),VEHICLE_TYPE_CODE_4-LIVERY VEHICLE,VEHICLE_TYPE_CODE_4-MOTORCYCLE,VEHICLE_TYPE_CODE_4-OTHER,VEHICLE_TYPE_CODE_4-PASSENGER VEHICLE,VEHICLE_TYPE_CODE_4-PICK-UP TRUCK,VEHICLE_TYPE_CODE_4-SMALL COM VEH(4 TIRES),VEHICLE_TYPE_CODE_4-SPORT UTILITY / STATION WAGON,VEHICLE_TYPE_CODE_4-TAXI,VEHICLE_TYPE_CODE_4-UNKNOWN,VEHICLE_TYPE_CODE_4-VAN,VEHICLE_TYPE_CODE_4-nan,VEHICLE_TYPE_CODE_5-BUS,VEHICLE_TYPE_CODE_5-CONTROL,VEHICLE_TYPE_CODE_5-LARGE COM VEH(6 OR MORE TIRES),VEHICLE_TYPE_CODE_5-LIVERY VEHICLE,VEHICLE_TYPE_CODE_5-MOTORCYCLE,VEHICLE_TYPE_CODE_5-OTHER,VEHICLE_TYPE_CODE_5-PASSENGER VEHICLE,VEHICLE_TYPE_CODE_5-PICK-UP TRUCK,VEHICLE_TYPE_CODE_5-SPORT UTILITY / STATION WAGON,VEHICLE_TYPE_CODE_5-TAXI,VEHICLE_TYPE_CODE_5-UNKNOWN,VEHICLE_TYPE_CODE_5-VAN,VEHICLE_TYPE_CODE_5-nan,Street_Condition,Traffic_Signal_Condition,Visibility,WetBulbFarenheit,WindSpeed,Precip,PrecipSum,intersectionID,total_involved
0,id-0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,10,21.666667,8.333333,0,-2.239962e-16,1,0
1,id-1,0,0,0,0,0,0,0,0,0,

In [11]:
d2.intersectionID.value_counts()

42840      219
30986      137
15968      135
22814      132
22556      116
21217      112
29303      111
21409      110
20585      105
22752      103
28915       96
21354       95
61999       92
49100       92
29315       91
44700       90
23799       90
42540       89
21340       88
22749       88
18762       87
20695       87
108320      86
29756       85
21460       85
9031585     84
21257       82
36072       82
22754       80
49787       80
          ... 
87468        1
87440        1
87453        1
87441        1
87442        1
87443        1
87444        1
87445        1
87446        1
87447        1
87448        1
87449        1
87450        1
87451        1
87452        1
87454        1
87467        1
87455        1
87456        1
87457        1
87458        1
87459        1
87460        1
87461        1
87462        1
87463        1
87464        1
87465        1
87466        1
1            1
Name: intersectionID, dtype: int64

In [ ]:
# d2.isnull().sum()
d2.to_csv('../data/vzwV2.csv', index=False)

In [ ]:
d2.shape